In [2]:
import numpy
import pandas as pd
from statistics import mean
from nltk.stem import WordNetLemmatizer
WNL = WordNetLemmatizer()
import nltk

In [3]:
data = pd.read_csv('negsents copy.csv')
data

,startphrase,ending1,ending2
0,She is honest,She can be trusted,She can not be trusted
1,she is dishonest,She can be trusted,She can not be trusted
2,The student was cooperative,He was helpful,he was not helpful
3,The student was uncooperative,He was helpful,he was not helpful
4,The man was stylish,The man was cool,The man was not cool
...,...,...,...
251,The illness was deadly,The illness is not serious,The illness is serious
252,The story was tedious,The story was not enthralling,The story was enthralling
253,The story was captivating,The story was not enthralling,The story was enthralling
254,That monster was terrifying,That monster was scary,That monster was not scary


In [3]:
data_edited = data[(data.labels!=-1)]
data_edited

,startphrase,ending1,ending2,labels,qid
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0,10
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1,10
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0,11
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1,11
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0,14
...,...,...,...,...,...
9105,Emails are as fast as a rocket,Emails are slow.,Emails are fast.,1,1817
9106,The dinner was as tasty as a leather shoe,The dinner was very dry and tasteless.,The dinner was award winning.,0,1819
9107,The dinner was as tasty as a Michelin star,The dinner was very dry and tasteless.,The dinner was award winning.,1,1819
9108,The explanation was like clear water,The explanation was easy and simple to understand,The explanation made to sense.,0,1821


# Lemmatise and save

In [4]:
def sent_lemmatize(sent):
    lemmasent = ''
    for w, tag in nltk.pos_tag(nltk.word_tokenize(sent.replace(",","").replace("it's","it is").replace("n't"," not").replace("'re'"," are").replace("'m"," am").replace("'s","").replace("-"," "))): # 
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = w
        else:
            lemma = WNL.lemmatize(w, wntag)
        lemmasent = lemmasent + lemma + ' '
    return lemmasent

In [11]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [13]:
# Create lem data
data = pd.read_csv('negsents copy.csv')
lemma_data = pd.DataFrame(columns = ['startphrase','ending1','ending2','labels','xid'])
for i, row in data.iterrows():
    lemmasent1 = sent_lemmatize(row[0])
    lemmasent2 = sent_lemmatize(row[1])
    lemmasent3 = sent_lemmatize(row[2])

    new_row = {'startphrase':lemmasent1, 'ending1':lemmasent2, 'ending2':lemmasent3, 'labels':row['labels'], 'xid':row['xid']}
    lemma_data = lemma_data.append(new_row, ignore_index=True)
    #new_row = [lemmasent1, lemmasent2, lemmasent3, row[3], row[4]]
    #lemma_data = pd.concat([lemma_data, new_row], axis=1)

lemma_data

,startphrase,ending1,ending2,labels,xid
0,She be honest,She can be trust,She can not be trust,0,10_0
1,she be dishonest,She can be trust,She can not be trust,1,10_1
2,The student be cooperative,He be helpful,he be not helpful,0,40_0
3,The student be uncooperative,He be helpful,he be not helpful,1,40_1
4,The man be stylish,The man be cool,The man be not cool,0,50_0
...,...,...,...,...,...
251,The illness be deadly,The illness be not serious,The illness be serious,1,X19_1
252,The story be tedious,The story be not enthral,The story be enthral,0,X30_0
253,The story be captivate,The story be not enthral,The story be enthral,1,X30_1
254,That monster be terrify,That monster be scary,That monster be not scary,0,X39_0


In [14]:
lemma_data.to_csv('negsents_lem.csv', index=False)

In [16]:
data = pd.read_csv('negsents copy.csv')
lemma_data = pd.DataFrame(columns = ['startphrase','ending1','ending2','labels','xid'])

def replacement(sent):
    return sent.replace(",","").replace(" '","'").replace("it's","it is").replace("n't"," not")
for i, row in data.iterrows():
    lemmasent1 = ''
    lemmasent2 = ''
    lemmasent3 = ''
    for w in nltk.word_tokenize(row[0]):
        lemmasent1 = lemmasent1 + WNL.lemmatize(w)+' '
    for w in nltk.word_tokenize(row[1]):
        lemmasent2 = lemmasent2 + WNL.lemmatize(w)+' '
    for w in nltk.word_tokenize(row[2]):
        lemmasent3 = lemmasent3 + WNL.lemmatize(w)+' '
    
    lemmasent1 = replacement(lemmasent1)
    lemmasent2 = replacement(lemmasent2)
    lemmasent3 = replacement(lemmasent3)
    
    new_row = {'startphrase':lemmasent1, 'ending1':lemmasent2, 'ending2':lemmasent3, 'labels':row[3], 'xid':row[4]}
    lemma_data = lemma_data.append(new_row, ignore_index=True)
    #new_row = [lemmasent1, lemmasent2, lemmasent3, row[3], row[4]]
    #lemma_data = pd.concat([lemma_data, new_row], axis=1)

lemma_data

,startphrase,ending1,ending2,labels,xid
0,She is honest,She can be trusted,She can not be trusted,0,10_0
1,she is dishonest,She can be trusted,She can not be trusted,1,10_1
2,The student wa cooperative,He wa helpful,he wa not helpful,0,40_0
3,The student wa uncooperative,He wa helpful,he wa not helpful,1,40_1
4,The man wa stylish,The man wa cool,The man wa not cool,0,50_0
...,...,...,...,...,...
251,The illness wa deadly,The illness is not serious,The illness is serious,1,X19_1
252,The story wa tedious,The story wa not enthralling,The story wa enthralling,0,X30_0
253,The story wa captivating,The story wa not enthralling,The story wa enthralling,1,X30_1
254,That monster wa terrifying,That monster wa scary,That monster wa not scary,0,X39_0


In [73]:
lemma_data.to_csv('data_lemmatised2.csv', index=False)